In [1]:
{- --------------------------------------------
   프로그래밍언어 (2020년 2학기 기말고사)
   
   이름: 홍길동
   학번: 99999999
   -------------------------------------------- -}

-- =========================================== --
-- 1번 문제 (10점)
{- 크기 순서로 비교가 가능한 원소들의 리스트를 받아
그 안의 원소들로 구성할 수 있는 가능한 모든 순서쌍
중에서 첫번째 원소가 두번째 원소보다 작은 경우만을
골라 리스트로 만들어주는 함수 sortedPairs를 작성하고
테스트를 2개 이상 작성하라.
예컨대, [2,1,3]로부터 [(1,2),(1,3),(2,3)]을 만들고,
[1,3,2,4]에서 [(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]를
만들어야 한다. 리스트에서 원소가 나타나는 순서는
관계없이 빠짐없고 중복없이 나타나기만 하면 된다.

힌트: 리스트 조건제시법을 활용하면 생각보다 간단히
작성할 수도 있다. [ (x,y) | x<-???, y<-???, ??? ]
 -}
sortedPairs ns = [ (x,y) | x<-ns, y<-ns, x < y ]

test1 = do
    putStrLn "### test1 ### ####################"
    -- write tests here
    print $ sortedPairs [2,1,3]
    print $ sortedPairs [1,3,2,4]


-- =========================================== --
-- 2번 문제 (20점)
data BTree a
  = BNull
  | BNode a (BTree a) (BTree a)
  deriving Show
{-
리스트에 대한 zip 함수와 비슷한 일을 이진트리(BTree)
데이터 타입에 대해 하는 zipBTree 함수를 정의해 보자.

두 개의 이진트리에서 서로 같은 곳에 위치한 노드의
데이터를 순서쌍으로 묶은 트리를 만들어내는 함수
zipBTree를 작성하고 두 개 이상의 테스트를 작성하라.
예컨대,
            2      y           (2,y)
zipBTree   / \    / \    =     /   \
          1   3  x   z     (1,x)   (3,z)

아래처럼 같은 위치에 노드가 없고 둘 중 한쪽이 비어
있으면 결과값 트리에서 그곳을 빈 트리로 처리한다.

            2      y           (2,y)
zipBTree   /      / \    =     /
          1      x   z     (1,x)
 -}
zipBTree :: BTree a -> BTree b -> BTree (a,b)
zipBTree _ BNull = BNull
zipBTree BNull _ = BNull
zipBTree (BNode x t1 t2) (BNode x' t1' t2') = BNode (x,x') (zipBTree t1 t1') (zipBTree t2 t2')
 
test2 = do
    putStrLn "### test2 ### ####################"
    -- write tests here
    print $ zipBTree tree1 tree2
    print $ zipBTree tree3 tree2

bLeaf x = BNode x BNull BNull

tree1 = BNode 2 (bLeaf 1) (bLeaf 3)
tree2 = BNode 'x' (bLeaf 'y') (bLeaf 'z')
tree3 = BNode 2 (bLeaf 1) BNull

-- =========================================== --
-- 3번 문제 (10점)
{- 리스트가 오름차순으로 정렬되어 있는지 검사하는
함수 sorted를 작성하고 테스트를 test2에 2개 이상
작성하라. 예컨대,
[1,2,3]와 [1,2,2,3]는 오름차순 정렬되어 있지만
[1,3,2]나 [2,1,2,3]은 그렇지 않다.
 -}
sorted :: Ord a => [a] -> Bool
sorted []  = True     -- 길이 0
sorted [x] = True     -- 길이 1
sorted (x:y:ys) = x <= y && sorted (y:ys)

test3 = do
    putStrLn "### test3 ### ####################"
    -- write tests here
    print $ sorted [1,2,3]
    print $ sorted [1,2,2,3]
    print $ sorted [1,3,2]
    print $ sorted [2,1,2,3]


-- =========================================== --
data Exp
  = Var Nm          -- x
  | Val Int         -- n
  | Add Exp Exp     -- e1 + e2
  | If Exp Exp Exp  -- if e then e1 else e2
  | Lam Nm Exp      -- \x.e
  | App Exp Exp     -- e1 e2
  | Rec Nm Exp      -- rec f e
  deriving Show

type Nm = String
data Value = Vint Int
           | Clos Env Exp
           deriving Show
type Env = [ (Nm, Value) ]

eval env (Var x) = lookup' x env
eval   _ (Val n) = Vint n
eval env (Add e1 e2) = Vint (n1 + n2)
  where
    n1 = fromVint (eval env e1)
    n2 = fromVint (eval env e2)
eval env (If e e1 e0) =
  eval env (if n/=0 then e1 else e0)
  where
    n = fromVint (eval env e)
eval env (Lam x e) = Clos env (Lam x e)
eval env (App e1 e2) =
  case v1 of  -- **** 4-(2) 함정: 아래 env를 env1으로 수정
    Clos env1 (Lam x e) -> eval ((x,v2):env1) e
    _ -> error $ show e1++" is not Clos"
  where
    v1 = eval env e1
    v2 = eval env e2
eval env (Rec f e) = eval env' e -- **** 4-(2)
  where
    env' = (f,Clos env' e) : env

lookup' x env =
  case lookup x env of
    Just v  -> v
    Nothing -> error $ x++" not found"

fromVint (Vint n) = n
fromVint _        = error "not Vint"


-- =========================================== --
-- 4번 문제 (70점)
{-
우리가 정의하고자 하는 프로그래밍 언어의 추상문법을
위와 같은 하스켈의 Exp 데이터 타입으로 정의하였다. 
우리가 정의하는 대상인 이 프로그래밍 언어에서
양의 정수값 k를 넘겨받으면 1부터 k까지 합을 구하는
재귀함수를 rec 문법으로 아래와 같이 작성할 수 있다.
rec f (\x.if x then 0 else x + f (x + (-1)))

*** 윗줄이 사실 잘못되어 있었는데 시험시간에 수정하지 못함 ***
rec f (\x.if x then x + f (x + (-1)) else 0)
*** 바로 위에처럼 then 과 else 식이 바뀌어야 함 **********

(1) 위의 재귀함수 함수를 Exp를 데이터 구조로 작성해
    아래 sumUpTo라는 이름에 지정하라 (부분점수 15점)
    
    *** 그래서 이 문제는 then 과 else 식의 순서를
    *** 그냥 주어진 그대로 혹은 잘못된 걸 알아서 바꿔도
    *** 둘 다 정답으로 인정함

(2) Rec의 경우에 대한 eval 함수를 작성하여 완성하고
    sumUpTo를 활용하여 eval로 실행하는 테스트를 2개
	이상 작성하라. (나머지 부분점수 55점)

    *** 이 문제는 시험에 주어졌던 then과 else가
    *** 바뀐 예제를 수정하지 않고 그냥 돌리면 무한루프
    *** 그러니까 이 부분까지 점수를 얻으려면 수정해서
    *** 테스트를 돌리던가 아니면 아예 새로운 예제를 작성해야
    *** 그리고 우연찮게 발동된 함정이 하나 있다 App의 코드가
    *** 잘못 작성되어 있는 곳을 수정해야 정상적으로 작동한다
 -}

-- -- 올바르게 수정된 것 기준으로 모범답안 작성
-- rec f (\x.if x then x + f (x + (-1)) else 0)
sumUpTo :: Exp
sumUpTo =
  Rec "f"
    (Lam "x"
      (If (Var "x")
        (Add
          (Var "x")
          (App (Var "f") (Var "x" `Add` Val (-1))))
        (Val 0) ) )

test4 = do
    putStrLn "### test4 ### ####################"
    -- write tests here
    print $ eval [] (App sumUpTo (Val 10))
    print $ eval [] (App sumUpTo (Val 100))


-- =========================================== --
-- 5번 문제 (50점)
{- 주어진 함수 f에 대해 정의역을 유한한
집합 xs으로 한정했을 때 g의 역상함수를
inverse g xs로 계산해내는 고차함수 inverse를
작성하라.

예컨대, 아래와 같이 5로 나눈 나머지를 구하는 함수
g의 역상 h를 정의역을 1부터 20까지의 정수로 한정해 
아래와 같이 정의했다면,

g :: Int -> Int
g = (`mod` 5)
h :: [Int] -> [Int]
h = inverse g [1..10]

이 때, h [1,4] = [1,4,6,9] 가 된다. h를 g의
역상이라 부르는 이유는 g 1 = 1, g 4 = 4,
g 6 = 1, g 9 = 4 이므로 h [1,4]의 결과값 집합의
모든 원소에 g를 적용했을 때 1이나 4로 계산되어
g의 치역에서 정의역 쪽으로 거슬러 올라간 값들만
나오게 되기 때문이다.

꼭 정수에서 정수로 가는 함수가 아닌 경우에도 
일반적으로 동작하도록 아래와 같은 타입으로
inverse를 작성하고 테스트를 2개 이상 작성하라.

*** 여기는 문제에서 제가 타입을 실수로 잘못 적어놓아서
*** 해결하는 데 의도한 것보다 괜히 좀 더 힘들어졌을 겁니다
 -}

-- -- ** 원래 타입대로는 안된다 (b가 비교가능 타입이어야) **
-- inverse ::  (a -> b) -> [a] -> ([b] -> [a])
-- -- ** 그래서 타입을 이렇게 바꾸거나
-- inverse :: Eq b => (a -> b) -> [a] -> ([b] -> [a])
-- -- ** 아니면 아예 지워버리면 된다 **
inverse f xs = \ ys -> [x | x<-xs, f x `elem` ys]
 
test5 = do
    putStrLn "### test5 ### ####################"
    -- write tests here
    print $ h [1,4]

g = (`mod` 5)
h = inverse g [1..10]

Line 232: Redundant lambda
Found:
inverse f xs = \ ys -> [x | x <- xs, f x `elem` ys]
Why not:
inverse f xs ys = [x | x <- xs, f x `elem` ys]

In [2]:
sortedPairs ns = [ (x,y) | x<-ns, y<-ns, x < y ]
sortedPairs [1,3,2,4,5]

[(1,3),(1,2),(1,4),(1,5),(3,4),(3,5),(2,3),(2,4),(2,5),(4,5)]